In [1]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re
import csv

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados para subir\\\\Dados Brutos\\\\INEP\\\\Adequação Formação Docente')


In [3]:
df = pd.read_csv('Tabela final de Adequação da Formação Docente.csv', sep=',')
df

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,EI_grupo1,...,EJA_EF_grupo1,EJA_EF_grupo2,EJA_EF_grupo3,EJA_EF_grupo4,EJA_EF_grupo5,EJA_EM_grupo1,EJA_EM_grupo2,EJA_EM_grupo3,EJA_EM_grupo4,EJA_EM_grupo5
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,--,...,--,--,--,--,--,--,--,--,--,--
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,...,45.6,0,39.8,10.7,3.9,61.5,0,33.7,0,4.8
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,--,...,--,--,--,--,--,--,--,--,--,--
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,--,...,--,--,--,--,--,--,--,--,--,--
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,100,...,--,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802862,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,100,...,--,--,--,--,--,--,--,--,--,--
1802863,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,26.8,...,--,--,--,--,--,--,--,--,--,--
1802864,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,100,...,--,--,--,--,--,--,--,--,--,--
1802865,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,75.5,...,--,--,--,--,--,--,--,--,--,--


In [4]:
df = df.replace('--', np.nan) #transformando os valores '--' como  NaN 

In [5]:
df.columns

Index(['Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município',
       'Código da Escola', 'Nome da Escola', 'Localização',
       'Dependência Administrativa', 'EI_grupo1', 'EI_grupo2', 'EI_grupo3',
       'EI_grupo4', 'EI_grupo5', 'EF_total_grupo1', 'EF_total_grupo2',
       'EF_total_grupo3', 'EF_total_grupo4', 'EF_total_grupo5',
       'EF_iniciais_grupo1', 'EF_iniciais_grupo2', 'EF_iniciais_grupo3',
       'EF_iniciais_grupo4', 'EF_iniciais_grupo5', 'EF_finais_grupo1',
       'EF_finais_grupo2', 'EF_finais_grupo3', 'EF_finais_grupo4',
       'EF_finais_grupo5', 'EM_grupo1', 'EM_grupo2', 'EM_grupo3', 'EM_grupo4',
       'EM_grupo5', 'EJA_EF_grupo1', 'EJA_EF_grupo2', 'EJA_EF_grupo3',
       'EJA_EF_grupo4', 'EJA_EF_grupo5', 'EJA_EM_grupo1', 'EJA_EM_grupo2',
       'EJA_EM_grupo3', 'EJA_EM_grupo4', 'EJA_EM_grupo5'],
      dtype='object')

In [24]:
transformar_float = ['EI_grupo1', 'EI_grupo2', 'EI_grupo3',
       'EI_grupo4', 'EI_grupo5', 'EF_total_grupo1', 'EF_total_grupo2',
       'EF_total_grupo3', 'EF_total_grupo4', 'EF_total_grupo5',
       'EF_iniciais_grupo1', 'EF_iniciais_grupo2', 'EF_iniciais_grupo3',
       'EF_iniciais_grupo4', 'EF_iniciais_grupo5', 'EF_finais_grupo1',
       'EF_finais_grupo2', 'EF_finais_grupo3', 'EF_finais_grupo4',
       'EF_finais_grupo5', 'EM_grupo1', 'EM_grupo2', 'EM_grupo3', 'EM_grupo4',
       'EM_grupo5', 'EJA_EF_grupo1', 'EJA_EF_grupo2', 'EJA_EF_grupo3',
       'EJA_EF_grupo4', 'EJA_EF_grupo5', 'EJA_EM_grupo1', 'EJA_EM_grupo2',
       'EJA_EM_grupo3', 'EJA_EM_grupo4', 'EJA_EM_grupo5'] #definindo uma variável para todas as colunas

In [25]:
df[transformar_float] = df[transformar_float].astype(float) #transformando todas as colunas em float 


In [26]:
df.dtypes #verificando 

Ano                             int64
Região                         object
UF                             object
Código do Município             int64
Nome do Município              object
Código da Escola                int64
Nome da Escola                 object
Localização                    object
Dependência Administrativa     object
EI_grupo1                     float64
EI_grupo2                     float64
EI_grupo3                     float64
EI_grupo4                     float64
EI_grupo5                     float64
EF_total_grupo1               float64
EF_total_grupo2               float64
EF_total_grupo3               float64
EF_total_grupo4               float64
EF_total_grupo5               float64
EF_iniciais_grupo1            float64
EF_iniciais_grupo2            float64
EF_iniciais_grupo3            float64
EF_iniciais_grupo4            float64
EF_iniciais_grupo5            float64
EF_finais_grupo1              float64
EF_finais_grupo2              float64
EF_finais_gr

In [27]:
#Padronizando todos os nomes das colunas
df = df.rename(columns= {'Ano':'ano', 
                        'Região':'nome_regiao', 
                        'UF': 'sigla_uf', 
                        'Código do Município':'id_municipio',
                        'Nome do Município':'nome_municipio',
                        'Código da Escola':'id_escola',
                        'Nome da Escola':'nome_escola',
                        'Localização':'localizacao',
                        'Dependência Administrativa':'dependencia_adm',
                        'EI_grupo1': 'ei_grupo1',
                        'EI_grupo2': 'ei_grupo2', 
                        'EI_grupo3': 'ei_grupo3',
                        'EI_grupo4': 'ei_grupo4', 
                        'EI_grupo5': 'ei_grupo5', 
                        'EF_total_grupo1': 'ef_total_grupo1', 
                        'EF_total_grupo2': 'ef_total_grupo2', 
                        'EF_total_grupo3': 'ef_total_grupo3', 
                        'EF_total_grupo4': 'ef_total_grupo4', 
                        'EF_total_grupo5': 'ef_total_grupo5',
                        'EF_finais_grupo1' : 'ef_finais_grupo1',
                        'EF_finais_grupo2' : 'ef_finais_grupo2', 
                        'EF_finais_grupo3' : 'ef_finais_grupo3', 
                        'EF_finais_grupo4' : 'ef_finais_grupo4',
                        'EF_finais_grupo5' : 'ef_finais_grupo5', 
                        'EM_grupo1' : 'em_grupo1', 
                        'EM_grupo2' : 'em_grupo2', 
                        'EM_grupo3' : 'em_grupo3', 
                        'EM_grupo4' : 'em_grupo4',
                        'EM_grupo5' : 'em_grupo5', 
                        'EJA_EF_grupo1' : 'eja_ef_grupo1',
                        'EJA_EF_grupo2' : 'eja_ef_grupo2', 
                        'EJA_EF_grupo3' : 'eja_ef_grupo3',
                        'EJA_EF_grupo4' : 'eja_ef_grupo4', 
                        'EJA_EF_grupo5' : 'eja_ef_grupo5', 
                        'EJA_EM_grupo1' : 'eja_em_grupo1', 
                        'EJA_EM_grupo2' : 'eja_em_grupo2',
                        'EJA_EM_grupo3' : 'eja_em_grupo3', 
                        'EJA_EM_grupo4' : 'eja_em_grupo4', 
                        'EJA_EM_grupo5' : 'eja_em_grupo5', 
                        'EF_iniciais_grupo1' : 'ef_iniciais_grupo1',
                        'EF_iniciais_grupo2' : 'ef_iniciais_grupo2', 
                        'EF_iniciais_grupo3' : 'ef_iniciais_grupo3',
                        'EF_iniciais_grupo4' : 'ef_iniciais_grupo4',
                        'EF_iniciais_grupo5' : 'ef_iniciais_grupo5' })
df

IndentationError: unexpected indent (1866110693.py, line 3)

In [ ]:
df = df.melt(id_vars =['ano', 'nome_regiao', 'sigla_uf', 'id_municipio','nome_municipio','id_escola','nome_escola','localizacao','dependencia_adm'], 
        value_vars =['ei_grupo1', 'ei_grupo2','ei_grupo3', 'ei_grupo4', 'ei_grupo5',
                     'ef_total_grupo1', 'ef_total_grupo2', 'ef_total_grupo3','ef_total_grupo4','ef_total_grupo5',
                     'ef_iniciais_grupo1','ef_iniciais_grupo2','ef_iniciais_grupo3','ef_iniciais_grupo4','ef_iniciais_grupo5',
                     'ef_finais_grupo1', 'ef_finais_grupo2','ef_finais_grupo3','ef_finais_grupo4','ef_finais_grupo5',
                     'em_grupo1', 'em_grupo2','em_grupo3','em_grupo4','em_grupo5',
                     'eja_ef_grupo1', 'eja_ef_grupo2', 'eja_ef_grupo3','eja_ef_grupo4', 'eja_ef_grupo5',
                     'eja_em_grupo1', 'eja_em_grupo2','eja_em_grupo3','eja_em_grupo4','eja_em_grupo5'], 
        var_name ='grupos_docentes_por_ano_escolar', 
        value_name ='per_valor')
df


,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,grupos_docentes_por_ano_escolar,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,ei_grupo1,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,ei_grupo1,NaN
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,ei_grupo1,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,ei_grupo1,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,ei_grupo1,100.0
...,...,...,...,...,...,...,...,...,...,...,...
63100340,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,eja_em_grupo5,NaN
63100341,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,eja_em_grupo5,NaN
63100342,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,eja_em_grupo5,NaN
63100343,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,eja_em_grupo5,NaN


In [28]:
df.columns

Index(['Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município',
       'Código da Escola', 'Nome da Escola', 'Localização',
       'Dependência Administrativa', 'EI_grupo1', 'EI_grupo2', 'EI_grupo3',
       'EI_grupo4', 'EI_grupo5', 'EF_total_grupo1', 'EF_total_grupo2',
       'EF_total_grupo3', 'EF_total_grupo4', 'EF_total_grupo5',
       'EF_iniciais_grupo1', 'EF_iniciais_grupo2', 'EF_iniciais_grupo3',
       'EF_iniciais_grupo4', 'EF_iniciais_grupo5', 'EF_finais_grupo1',
       'EF_finais_grupo2', 'EF_finais_grupo3', 'EF_finais_grupo4',
       'EF_finais_grupo5', 'EM_grupo1', 'EM_grupo2', 'EM_grupo3', 'EM_grupo4',
       'EM_grupo5', 'EJA_EF_grupo1', 'EJA_EF_grupo2', 'EJA_EF_grupo3',
       'EJA_EF_grupo4', 'EJA_EF_grupo5', 'EJA_EM_grupo1', 'EJA_EM_grupo2',
       'EJA_EM_grupo3', 'EJA_EM_grupo4', 'EJA_EM_grupo5'],
      dtype='object')

Subindo para o GBQ

In [ ]:
##client = bigquery.Client()

In [ ]:
##dataset_ref = client.dataset('percepcao_mensuracao_qualidade')